In [ ]:
import pandas as pd
import numpy as np

from multiprocessing import Pool, cpu_count
!pip install python-whois
import whois
import re
!pip install tldextract
import tldextract
from urllib.parse import urlparse
from datetime import datetime
from tqdm import tqdm
import time

import multiprocessing

import ssl
import socket
from datetime import timezone
import requests
import time
from urllib.parse import urlparse, parse_qs
import idna
import unicodedata

from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.neighbors import KNeighborsClassifier


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.6 MB/s eta 0:00:00


In [ ]:
columns=[
#total url related parameters
    'URL','URL_Length','Url_Shortening','Ssl_Info',
    #"""'AsNo',"""
    'Global_Ranking','Has_Suspicious_Params','Num_Languages',
#Scheme
    'Uses_HTTPS',
#User Info	user
    'Has_User_Info','User_Info_Length',
#Subdomain
    'Num_Subdomains',"Num_Digits_Subdomain",'Num_Hyphens_Subdomain','Length_Subdomain','Num_Dots_Subdomain','Num_Dash_Subdomain',
    'Num_AtSymbol_Subdomain','Num_TildeSymbol_Subdomain','Num_Underscore_Subdomain','Num_Percent_Subdomain','Num_Ampersand_Subdomain', 'Num_Hash_Subdomain',
    #"""'DomainInSubdomains',"""
          'Hex_Encoded_Characters_Subdomain',
#Domain
    'Has_Ip',"Num_Digits_Domain",'Num_Hyphens_Domain','Length_Domain','Num_Dots_Domain','Num_Dash_Domain','Num_AtSymbol_Domain',
    'Domain_Age','Num_TildeSymbol_Domain','Num_Underscore_Domain','Num_Percent_Domain','Num_Ampersand_Domain','Num_Hash_Domain', 'Hex_Encoded_Characters_Domain',
#TLD
    'TLD',
#Port
    'Has_Port',
#Path
    'Path_Level',"Num_Digits_Path",'Num_Hyphens_Path','Length_Path','Num_Dots_Path','Num_Dash_Path','Num_AtSymbol_Path',
    'Num_TildeSymbol_Path','Num_Underscore_Path','Num_Percent_Path','Num_Ampersand_Path','Num_Hash_Path','Num_DoubleSlash_Path',
#"""'DomainInPaths',"""
    'Hex_Encoded_Characters_Path',
#Query
    "Num_Digits_Query",'Num_Hyphens_Query','Length_Query','Num_Query_Params','Num_Dots_Query','Num_Dash_Query','Num_AtSymbol_Query',
    'Num_TildeSymbol_Query','Num_Underscore_Query','Num_Percent_Query','Num_Ampersand_Query','Num_Hash_Query','Hex_Encoded_Characters_Query',
#Fragment
    "Has_Fragment"
]

In [ ]:
suspicious_url_parameters = {
    # Open Redirects & Phishing
    "redirect", "next", "url", "dest", "out", "view", "link", "goto", "target", "forward", "jump",

    # Remote Code Execution (RCE)
    "cmd", "eval", "execute", "exec", "shell", "run", "code", "command", "process", "do",

    # File Inclusion & Download Attacks (LFI & RFI)
    "file", "filepath", "download", "include", "load", "document", "pdf", "attachment",
    "export", "import", "module", "dir", "read", "write", "stream", "content", "getfile",

    # Download-related extensions
    "exe", "msi", "apk", "bat", "sh", "cmd", "jar", "scr", "zip", "rar", "7z", "tar", "gz",
    "pdf", "doc", "docx", "xls", "xlsx", "ppt", "pptx", "iso", "img", "vbs", "ps1", "dll", "reg",

    # SQL Injection (SQLi)
    "id", "query", "search", "order", "sort", "filter", "select", "drop", "union", "table",
    "from", "where", "update", "delete", "insert", "fetch", "database", "schema",

    # Authentication Bypass & Session Hijacking
    "token", "session", "auth", "jwt", "sso", "apikey", "password", "pass", "user", "username",
    "email", "key", "credential", "login", "logout", "csrf", "remember", "access", "validate",

    # XSS (Cross-Site Scripting)
    "script", "alert", "onload", "onerror", "onmouseover", "onclick", "onfocus",
    "data", "src", "href", "javascript", "vbscript", "expression", "cookie", "input",

    # Command Injection
    "ping", "nc", "ncat", "curl", "wget", "bash", "sh", "zsh", "perl", "python",

    # Path Traversal & Directory Access
    "path", "dir", "folder", "root", "home", "var", "etc", "passwd", "config", "secret",

    # API & Debugging
    "debug", "test", "trace", "log", "verbose", "info", "error", "status", "diagnostic"
}


In [ ]:
import requests
from datetime import datetime

BASE_URL = "https://tranco-list.eu/api"

def get_latest_tranco_rank(domain):
    url = f"{BASE_URL}/ranks/domain/{domain}"
    response = requests.get(url)

    if response.status_code == 200:
        ranks = response.json().get("ranks", [])

        if not ranks:
            return 0

        ranks.sort(key=lambda x: x["date"], reverse=True)

        return ranks[0]["rank"]

    elif response.status_code == 403:
        return -1
    elif response.status_code == 429:
        return -1
    else:
        return -1



In [ ]:
def decode_idn(domain):
    """Decode multi-layer IDN domains (handles double Punycode encoding)."""
    while True:
        try:
            decoded = idna.decode(domain)
            if decoded == domain:
                break
            domain = decoded
        except idna.IDNAError:
            break
    return domain
IGNORE_CHARACTERS = [
    ".", "-", "_", "~", "/", "?", "#", "&", "=", "%", "@", ":", ";", ",", "+", "*", "!", "$", "'", '"', "(", ")", "[", "]", "{", "}", "|", "\\"
]
def get_num_languages(domain):
    """Decode domain, identify character scripts, and count unique ones."""

    decoded_domain = decode_idn(domain)

    scripts = set()

    for char in decoded_domain:
        if char in IGNORE_CHARACTERS:
            continue

        try:
            script_name = unicodedata.name(char).split()[0]  # Get script category
            scripts.add(script_name)
        except ValueError:
            continue

    return len(scripts)

In [ ]:
def analyze_string_metrics(input_str):
    """Returns a dictionary with counts of various special characters in the input string."""
    return {
        "Num_Digits": len(re.findall(r"\d", input_str)),  # Count digits (0-9)
        "Num_Hyphens": input_str.count("-"),  # Count hyphens (-)
        "Length": len(input_str),  # Total length of the string
        "Num_Dots": input_str.count("."),  # Count dots (.)
        "Num_Dash": input_str.count("_"),  # Count underscores (_)
        "Num_AtSymbol": input_str.count("@"),  # Count at symbols (@)
        "Num_TildeSymbol": input_str.count("~"),  # Count tilde (~)
        "Num_Underscore": input_str.count("_"),  # Count underscores (_)
        "Num_Percent": input_str.count("%"),  # Count percent symbols (%)
        "Num_Ampersand": input_str.count("&"),  # Count ampersands (&)
        "Num_Hash": input_str.count("#"),  # Count hash symbols (#)
        "Hex_Encoded_Characters": len(re.findall(r"%[0-9A-Fa-f]{2}", input_str))  # Count hex-encoded characters
    }

In [ ]:
def process_Subdomain(subdomain):
    num_subdomains = len([sub for sub in subdomain.split(".") if sub])
    if subdomain.count('.')==0:
        num_subdomains=0
        fetures=analyze_string_metrics("")
    else:
        subdomain= '.'.join(subdomain.split('.')[:-1])
        fetures=analyze_string_metrics(subdomain)
    fetures = {f"{key}_Subdomain": value for key, value in fetures.items()}
    fetures['Num_Subdomains']=num_subdomains
    return fetures

In [ ]:
def process_Domain(domain,suffix):
    fetures=analyze_string_metrics(domain)
    fetures = {f"{key}_Domain": value for key, value in fetures.items()}
    fetures['Domain_Age'] = get_domain_age(domain+'.'+suffix)
    fetures['Has_Ip'] = bool(re.match(r"\d+\.\d+\.\d+\.\d+", domain))
    return fetures

In [ ]:
def process_Path(path):
    fetures={}
    fetures=analyze_string_metrics(path)
    fetures = {f"{key}_Path": value for key, value in fetures.items()}
    if path=='':
        fetures['Path_Level']=0
    elif path=='/':
        fetures['Path_Level']=1
    else:
        fetures['Path_Level']=path.count('/')
    fetures['Num_DoubleSlash_Path']=path.count('//')
    return fetures


In [ ]:
def process_Query(query):
    fetures={}
    fetures=analyze_string_metrics(query)
    fetures = {f"{key}_Query": value for key, value in fetures.items()}
    fetures['Num_Query_Params'] = len(parse_qs(query))
    return fetures

In [ ]:
def get_domain_age(domain):
    for _ in range(3):
        try:
            domain_info = whois.whois(domain)
            creation_date = domain_info.creation_date

            if isinstance(creation_date, list):
                creation_date = [
                    d.replace(tzinfo=timezone.utc) if d.tzinfo is None else d.astimezone(timezone.utc)
                    for d in creation_date if isinstance(d, datetime)
                ]
                if not creation_date:
                    continue
                creation_date = min(creation_date)


            if isinstance(creation_date, datetime):
                if creation_date.tzinfo is None:
                    creation_date = creation_date.replace(tzinfo=timezone.utc)
                else:
                    creation_date = creation_date.astimezone(timezone.utc)

                return (datetime.now(timezone.utc) - creation_date).days

        except Exception as e:
            print(f"Lookup failed for {domain}: {e}")
            time.sleep(1)

    return -1

In [ ]:
def get_user_info(url):
    parsed_url = urlparse(url)
    username = parsed_url.username or ""
    password = parsed_url.password or ""

    has_user_info = bool(username or password)
    user_info_length = len(username) + len(password)

    return has_user_info, user_info_length

In [ ]:
def has_fragment(url):
    parsed_url = urlparse(url)
    return bool(parsed_url.fragment)

In [ ]:
shorteners = eval(open('/content/url_shortener_extensions.txt','r').read())

def is_shortened_url(url):
    from urllib.parse import urlparse
    domain = urlparse(url).netloc.lower()
    return domain in shorteners

In [ ]:
def has_port(url):
    """Returns True if the URL has an explicit port, otherwise False."""
    parsed_url = urlparse(url)
    return parsed_url.port is not None

In [ ]:
def get_ssl_certificate_info(hostname, port=443):
    # 1 - valid, 0 - invalid, -1 - error
    try:
        context = ssl.create_default_context()
        with socket.create_connection((hostname, port), timeout=5) as sock:
            with context.wrap_socket(sock, server_hostname=hostname) as ssock:
                cert = ssock.getpeercert()

                if not cert:
                    return 0

                # Extract issuer and subject
                issuer = dict(x[0] for x in cert.get("issuer", []))
                subject = dict(x[0] for x in cert.get("subject", []))

                # Extract expiry date and convert it to timezone-aware
                expiry_date = datetime.strptime(cert["notAfter"], "%b %d %H:%M:%S %Y %Z")
                expiry_date = expiry_date.replace(tzinfo=timezone.utc)  # Convert to offset-aware datetime

                is_expired = expiry_date < datetime.now(timezone.utc)

                # Check if self-signed
                is_self_signed = issuer == subject

                # Extract SAN (Subject Alternative Name)
                san_list = [ext[1] for ext in cert.get("subjectAltName", []) if ext[0] == "DNS"]

                # Validate hostname against CN and SANs
                valid_hostname = hostname in san_list or hostname == subject.get("commonName", "Unknown")

                # Issuer should be checked against a valid list of certificate authorities
                if valid_hostname and (not is_expired and not is_self_signed):
                    return 1
                return 0
    except ssl.SSLCertVerificationError:
        return 0
    except Exception as e:
        print(f"SSL check error for {hostname}: {e}")
        return -1

In [ ]:
def has_fragment(url):
    """Check if a URL contains a fragment (#)"""
    parsed_url = urlparse(url)
    return bool(parsed_url.fragment)

In [ ]:

multiprocessing.set_start_method("spawn", force=True)

def process_url(url):
    try:
        fetures = {}
        fetures['URL'] = url
        fetures['URL_Length'] = len(url)
        fetures['Url_Shortening'] = is_shortened_url(url)

        parsed = urlparse(url)
        domain_info = tldextract.extract(url)
        domain = f"{domain_info.domain}.{domain_info.suffix}"

        fetures['Ssl_Info'] = None#get_ssl_certificate_info(domain)
        fetures['Global_Ranking'] = get_latest_tranco_rank(domain)

        query_params = parsed.query.split("&") if parsed.query else []
        fetures['Has_Suspicious_Params'] = any(param.split("=")[0] in suspicious_url_parameters for param in query_params)

        fetures['Num_Languages'] = get_num_languages(domain)
        fetures['Uses_HTTPS'] = 1 if url.startswith("https") else 0
        fetures['Has_User_Info'], fetures['User_Info_Length'] = get_user_info(url)

        subdomain_info = process_Subdomain(domain_info.subdomain)
        fetures.update(subdomain_info)

        Domain_info = process_Domain(domain_info.domain, domain_info.suffix)
        fetures.update(Domain_info)

        fetures['TLD'] = domain_info.suffix
        fetures['Has_Port'] = has_port(url)

        path_info = process_Path(parsed.path)
        fetures.update(path_info)

        query_info = process_Query(parsed.query)
        fetures.update(query_info)

        fetures['Has_Fragment'] = has_fragment(url)

        return fetures

    except Exception as e:
        # Optionally log the error
        # print(f"Error processing {url}: {e}")
        return None

def extract_url_features_parallel(urls, num_workers=None):
    num_workers = min(num_workers or cpu_count(), 8)  # Optimize for I/O tasks

    with ThreadPool(num_workers) as pool:  # Thread-based pool for speedup
        data = list(tqdm(pool.imap(process_url, urls), total=len(urls), desc="Processing URLs"))

    # Remove None values (failed URLs)
    data = [entry for entry in data if entry]

    # Ensure each row has all expected columns
    features = [{col: entry.get(col, None) if isinstance(entry, dict) else None for col in columns} for entry in data]

    return pd.DataFrame(features, columns=columns)


In [ ]:
# Load your phishtank database
phistankDB = pd.read_csv('/content/phistank.csv')

phistankDB = phistankDB.sample(n=22000, random_state=42)['url']
phistankDB.to_csv('22000_pt.csv', index=False)
# Get the URL list (from the column index 1)
#url_list = phistankDB.iloc[:, 1].tolist()
#print(len(url_list))

In [ ]:
t1m = pd.read_csv('/content/top-1m.csv')
t1m = t1m.sample(n=22000, random_state=42).iloc[:,[1]]
t1m.columns=['url']
t1m.to_csv('22000_1m.csv', index=False)

In [ ]:
phistankDB = pd.read_csv('/content/22000_1m.csv')
url_list = phistankDB.iloc[:, 0].tolist()

In [ ]:
# Process a chunk (e.g., from index 4000 to 6000)
df = extract_url_features_parallel(url_list[10000:22000])
df.to_csv('top1m(10000_22000).csv', index=False)

Processing URLs: 100%|██████████| 12000/12000 [7:37:26<00:00,  2.29s/it]


In [ ]:
phistankDB = pd.read_csv('/content/phistank.csv')
phistankDB = phistankDB.sample(n=10000, random_state=42)
url_list = phistankDB.iloc[:,1].tolist()
df = extract_url_features_parallel(url_list[4000:6000])
df.to_csv('top1m(4000:6000).csv', index=False)

In [ ]:
#valisarion set
import pandas as pd
import random

phistankDB = pd.read_csv('/content/phistank.csv')
t1m = pd.read_csv('/content/top-1m.csv')

url_list_pt = phistankDB.iloc[:, 1].tolist()
url_list_1m = t1m.iloc[:, 1].tolist()

already_used = pd.concat([pd.read_csv('/content/combined_phishtank.csv'), pd.read_csv('/content/combined_top1m.csv')], ignore_index=True)
already_used_urls = set(already_used['URL'])  # assuming column is named 'URL'

filtered_urls_pt = [url for url in url_list_pt if url not in already_used_urls]
filtered_urls_1m = [url for url in url_list_1m if url not in already_used_urls]

# Sample 200 from each
selected_urls_pt = random.sample(filtered_urls_pt, 200)
selected_urls_1m = random.sample(filtered_urls_1m, 200)

# Feature extraction
df_pt = extract_url_features_parallel(selected_urls_pt)
df_1m = extract_url_features_parallel(selected_urls_1m)

# Add labels
df_pt['label'] = 1
df_1m['label'] = 0

# Combine and save
df = pd.concat([df_pt, df_1m], ignore_index=True)
df.to_csv('validation_ds.csv', index=False)


Processing URLs: 100%|██████████| 200/200 [07:15<00:00,  2.18s/it]


In [ ]:
#expanding the dataset

import pandas as pd
import random
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

start=5000
end=10000


phistankDB = pd.read_csv('/content/phistank.csv')
t1m = pd.read_csv('/content/top-1m.csv')

url_list_pt = phistankDB.iloc[:, 1].tolist()
url_list_1m = t1m.iloc[:, 1].tolist()

already_used = pd.concat([pd.read_csv('/content/combined_phishtank.csv'), pd.read_csv('/content/combined_top1m.csv')], ignore_index=True)
already_used_urls = set(already_used['URL'])  # assuming column is named 'URL'

filtered_urls_pt = [url for url in url_list_pt if url not in already_used_urls]
filtered_urls_1m = [url for url in url_list_1m if url not in already_used_urls]


#selected_urls_pt = random.sample(filtered_urls_pt, 200)
#selected_urls_1m = random.sample(filtered_urls_1m, 200)
random.seed(42)
random.shuffle(filtered_urls_pt)
random.seed(42)
random.shuffle(filtered_urls_1m)


# Feature extraction
df_pt = extract_url_features_parallel(filtered_urls_pt[start:end])
df_1m = extract_url_features_parallel(filtered_urls_1m[start:end])

# Add labels
df_pt['label'] = 1
df_1m['label'] = 0

# File paths in Google Drive (update path if needed)
output_folder = '/content/drive/MyDrive/URL_Feature_Sets/'
file_pt = f'{output_folder}phistank({start+22000}_{end+22000}).csv'
file_1m = f'{output_folder}top1m({start+22000}_{end+22000}).csv'

# Save directly to Drive
df_pt.to_csv(file_pt, index=False)
df_1m.to_csv(file_1m, index=False)